In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
def RSDE(W, k=2, lambda_val=0.01, maxiter=100 , labels = None , only_show_last = True):
    nn, mm = W.shape

    # Initialization
    B = np.random.rand(nn, k)
    M = np.random.rand(nn, k)
    Q = np.eye(k)

    for i in range(maxiter):
        # Update G
        G = calculateG(B, Q, 0, 1)

        # Update B
        AB = np.dot(W, M) + lambda_val * np.dot(G, Q)
        LB, bb, RB = np.linalg.svd(AB, full_matrices=False)
        B = np.dot(LB, RB)
        
        if (not(labels is None)):
            if (not only_show_last) or (i == maxiter-1):
                # Create a scatter plot of the PCA-transformed data
                plt.scatter(B[:, 0], B[:, 1], c=labels, cmap='viridis')
                plt.xlabel('RSDE 1')
                plt.ylabel('RSDE 2')
                plt.title('RSDE with 2 Components iter = '+ str(i+1))
                plt.show()
        
        # Update M
        M = np.dot(W.T, B)

        # Update Q
        AQ = np.dot(G.T, B)
        LQ, qq, RQ = np.linalg.svd(AQ, full_matrices=False)
        Q = np.dot(LQ, RQ)

    return B, G, M

def calculateG(F, H, a, BQ):
    n, class_num = F.shape
    T = np.zeros((n, class_num))

    for i in range(class_num):
        te = F - np.ones((n, 1)) * H[i, :]
        aa = np.sum(te * te, axis=1)
        T[:, i] = aa

    T = T - a * 2 * BQ
    idx = np.argmin(T, axis=1)
    G1 = np.zeros((n, class_num))

    for i in range(n):
        G1[i, idx[i]] = 1

    return G1